# Carregamento da Base

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Carregar a base e visualizar primeiras linhas
df1 = pd.read_csv('bank.csv')  #header=-1 indica que não há linha de header
df1.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [3]:
#dimnesões da base
df1.shape

(11162, 17)

# Converte atributos categóricos em binário

In [5]:
#!pip install category-encoders

In [6]:
df1.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
deposit      object
dtype: object

In [7]:
categorical_columns = df1.select_dtypes(include=['object'])
categorical_columns = categorical_columns.columns
categorical_columns = categorical_columns[categorical_columns != 'deposit']
categorical_columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

In [8]:
import category_encoders as ce
encoder = ce.BinaryEncoder()
df_binary = encoder.fit_transform(df1.loc[:,categorical_columns])
df_binary.head()

,job_0,job_1,job_2,job_3,marital_0,marital_1,education_0,education_1,education_2,default_0,...,loan_1,contact_0,contact_1,month_0,month_1,month_2,month_3,poutcome_0,poutcome_1,poutcome_2
0,0,0,0,1,0,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,1
1,0,0,0,1,0,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,1
2,0,0,1,0,0,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,1
3,0,0,1,1,0,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,1
4,0,0,0,1,0,1,0,1,0,0,...,1,0,1,0,0,0,1,0,0,1


# Divisão em treino e teste

In [9]:
int_columns = df1.select_dtypes(include=['int'])
int_columns = int_columns.columns.values
columns = np.append(int_columns, 'deposit')
columns

array(['age', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous', 'deposit'], dtype=object)

In [10]:
df1 = pd.concat([df_binary, df1.loc[:,columns]], axis=1)
df1.head()

,job_0,job_1,job_2,job_3,marital_0,marital_1,education_0,education_1,education_2,default_0,...,poutcome_1,poutcome_2,age,balance,day,duration,campaign,pdays,previous,deposit
0,0,0,0,1,0,1,0,0,1,0,...,0,1,59,2343,5,1042,1,-1,0,yes
1,0,0,0,1,0,1,0,0,1,0,...,0,1,56,45,5,1467,1,-1,0,yes
2,0,0,1,0,0,1,0,0,1,0,...,0,1,41,1270,5,1389,1,-1,0,yes
3,0,0,1,1,0,1,0,0,1,0,...,0,1,55,2476,5,579,1,-1,0,yes
4,0,0,0,1,0,1,0,1,0,0,...,0,1,54,184,5,673,2,-1,0,yes


In [11]:
import random
np.random.seed(1) #semente inicial
nlinhas = df1.shape[0]

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df1.loc[:,'job_0':'previous'], df1.loc[:,'deposit'], test_size=0.2)

In [13]:
y_test.value_counts()

no     1165
yes    1068
Name: deposit, dtype: int64

# Normalização

In [14]:
from sklearn import preprocessing
preprocessParams = preprocessing.StandardScaler().fit(x_train)

In [15]:
X_train_normalized = preprocessParams.transform(x_train)
X_test_normalized = preprocessParams.transform(x_test)

# Treinamento da Rede Neural

In [16]:
from keras import Sequential
from keras.layers import Dense

In [17]:
x_train.shape

(8929, 31)

In [18]:
RN = Sequential()
RN.add(Dense(22,input_shape = X_train_normalized.shape[1:], activation = 'sigmoid'))
RN.add(Dense(10, activation = 'sigmoid'))
RN.add(Dense(2,activation = 'sigmoid'))
RN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 22)                704       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                230       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 956
Trainable params: 956
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Transformação Dummy
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown = 'ignore')
encoder.fit(pd.DataFrame(y_train))

y_train = encoder.transform(pd.DataFrame(y_train)).toarray()
y_test = encoder.transform(pd.DataFrame(y_test)).toarray()

In [20]:
y_train

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [21]:
RN.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['accuracy'])
history = RN.fit(X_train_normalized,y_train, epochs = 100, validation_split=0.2) 

#Testar rede
score = RN.evaluate(X_test_normalized, y_test, verbose = 0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

2022-04-10 17:56:17.374513: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
224/224 [==============================] - 1s 3ms/step - loss: 0.2993 - accuracy: 0.4768 - val_loss: 0.2869 - val_accuracy: 0.4563
Epoch 2/100
224/224 [==============================] - 0s 2ms/step - loss: 0.2690 - accuracy: 0.4768 - val_loss: 0.2635 - val_accuracy: 0.4563
Epoch 3/100
224/224 [==============================] - 0s 2ms/step - loss: 0.2554 - accuracy: 0.4768 - val_loss: 0.2536 - val_accuracy: 0.4563
Epoch 4/100
224/224 [==============================] - 0s 2ms/step - loss: 0.2503 - accuracy: 0.4799 - val_loss: 0.2498 - val_accuracy: 0.4793
Epoch 5/100
224/224 [==============================] - 0s 2ms/step - loss: 0.2484 - accuracy: 0.5568 - val_loss: 0.2480 - val_accuracy: 0.5997
Epoch 6/100
224/224 [==============================] - 0s 2ms/step - loss: 0.2476 - accuracy: 0.6062 - val_loss: 0.2471 - val_accuracy: 0.6013
Epoch 7/100
224/224 [==============================] - 0s 2ms/step - loss: 0.2471 - accuracy: 0.5922 - val_loss: 0.2464 - val_accuracy: 0.5868

Epoch 58/100
224/224 [==============================] - 0s 2ms/step - loss: 0.1976 - accuracy: 0.7528 - val_loss: 0.1985 - val_accuracy: 0.7475
Epoch 59/100
224/224 [==============================] - 0s 2ms/step - loss: 0.1960 - accuracy: 0.7554 - val_loss: 0.1970 - val_accuracy: 0.7497
Epoch 60/100
224/224 [==============================] - 0s 2ms/step - loss: 0.1944 - accuracy: 0.7563 - val_loss: 0.1955 - val_accuracy: 0.7514
Epoch 61/100
224/224 [==============================] - 0s 2ms/step - loss: 0.1928 - accuracy: 0.7578 - val_loss: 0.1940 - val_accuracy: 0.7531
Epoch 62/100
224/224 [==============================] - 0s 2ms/step - loss: 0.1912 - accuracy: 0.7626 - val_loss: 0.1925 - val_accuracy: 0.7542
Epoch 63/100
224/224 [==============================] - 0s 2ms/step - loss: 0.1896 - accuracy: 0.7626 - val_loss: 0.1910 - val_accuracy: 0.7542
Epoch 64/100
224/224 [==============================] - 0s 2ms/step - loss: 0.1881 - accuracy: 0.7642 - val_loss: 0.1896 - val_accuracy:

In [ ]:
# Gráfico do treinamento: custo do treino e validação
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Loss de treino e validação')
plt.ylabel('loss')
plt.xlabel('época')
plt.legend();

# Avaliando resultados

In [ ]:
# Vamos testar a rede
from sklearn.metrics import confusion_matrix
y_test_predicted = RN.predict(X_test_normalized)
y_test_predicted_indexes = np.argmax(y_test_predicted,axis=1)
y_test_indexes = np.argmax(y_test, axis=1)

In [ ]:
#matriz de confusão
confMatrix = confusion_matrix(y_test_predicted_indexes, y_test_indexes) 
confMatrix

In [ ]:
# Matriz de confusão
confMatrix = confusion_matrix(y_test_predicted_indexes, y_test_indexes)
pd.DataFrame(confMatrix, 
             index=['Pred No', 'Pred Yes'], 
             columns=['Real No', 'Real Yes'],) #matriz de confusão

In [ ]:
import seaborn as sns
ax = plt.subplot()
sns.heatmap(confMatrix, annot=True, fmt=".0f", cmap=plt.cm.Blues)
plt.xlabel('Real')
plt.ylabel('Previsto')
plt.title('Matriz de Confusão')

# Colocar os nomes
ax.xaxis.set_ticklabels(['No', 'Yes']) 
ax.yaxis.set_ticklabels(['No', 'Yes'])
plt.show()